# 🧪 The GFP "Molecular Forge"
### Witnessing the Birth of Biological Light

---

## 🎯 What You'll Learn

The **Green Fluorescent Protein (GFP)** revolutionized cell biology by enabling scientists to watch living processes in real-time. Its discovery earned the 2008 Nobel Prize in Chemistry.

**What makes GFP special?**

Unlike other fluorescent molecules that must be added to cells, GFP **creates its own chromophore** through an autocatalytic process. Three amino acids (`SER65-TYR66-GLY67`) spontaneously rearrange to form a fluorescent π-conjugated system.

**In this tutorial:**
- 🔬 Visualize the SER-TYR-GLY motif before maturation
- ⚗️ Walk through the 3-step maturation: Cyclization → Dehydration → Oxidation
- 🌈 Understand the spectroscopy: why green? (λ_max ≈ 508 nm)
- 🧬 See how π-conjugation creates fluorescence

> **💡 Nobel Prize Context**: Osamu Shimomura, Martin Chalfie, and Roger Tsien shared the 2008 Nobel Prize "for the discovery and development of the green fluorescent protein, GFP."

---

In [ ]:
# 🔧 Environment Detection & Setup
import sys, os
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('🌐 Running in Google Colab')
    try:
        import synth_pdb
        print('   ✅ synth-pdb already installed')
    except ImportError:
        print('   📦 Installing synth-pdb...')
        !pip install -q synth-pdb
        print('   ✅ Installation complete')
    import plotly.io as pio
    pio.renderers.default = 'colab'
else:
    print('💻 Running in local Jupyter environment')
    sys.path.append(os.path.abspath('../../'))

print('✅ Environment configured!')

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, SelectionSlider
import numpy as np
import py3Dmol
import plotly.graph_objects as go
from synth_pdb import PeptideGenerator
import biotite.structure as struc
from biotite.structure.io.pdb import PDBFile
import tempfile

print('✅ GFP Molecular Forge Ready!')

## 📚 Chromophore Chemistry & Spectroscopy

### The Maturation Mechanism

GFP's chromophore forms through three autocatalytic steps:

#### Step 1: Cyclization (Nucleophilic Attack)

The **glycine-67 backbone nitrogen** attacks the **serine-65 carbonyl carbon**, forming a 5-membered imidazolinone ring.

```
Ser65-Tyr66-Gly67  →  [Cyclic intermediate]
```

**Mechanism**: Nucleophilic addition-elimination
- Gly67 N: acts as nucleophile
- Ser65 C=O: electrophilic carbonyl
- Result: 5-membered heterocycle

#### Step 2: Dehydration

Water is eliminated from the Tyr66 Cα, creating a **double bond** that extends conjugation.

```
[Cyclic intermediate] - H₂O  →  [Dehydrated intermediate]
```

**Result**: Cα=Cβ double bond in Tyr66

#### Step 3: Oxidation

Molecular oxygen (O₂) oxidizes the Tyr66 Cα-Cβ bond, completing the **π-conjugated system**.

```
[Dehydrated intermediate] + O₂  →  [Mature chromophore] + H₂O₂
```

**Result**: Fully conjugated p-hydroxybenzylidene-imidazolinone

---

### Why Green? Spectroscopy Fundamentals

**Absorption and Emission**

The mature chromophore has:
- **Absorption maximum**: λ_abs ≈ 395 nm (UV) and 475 nm (blue)
- **Emission maximum**: λ_em ≈ 508 nm (green)
- **Stokes shift**: ~33 nm

**The Physics**

Fluorescence occurs through electronic transitions:

1. **Absorption**: Blue photon (475 nm) excites electron from HOMO → LUMO
2. **Vibrational relaxation**: Energy dissipates as heat
3. **Emission**: Electron returns to ground state, emitting green photon (508 nm)

**Energy relationship**:

$$E = h\nu = \frac{hc}{\lambda}$$

Where:
- E = photon energy
- h = Planck's constant (6.626 × 10⁻³⁴ J·s)
- c = speed of light (3 × 10⁸ m/s)
- λ = wavelength

**HOMO-LUMO Gap**

The π-conjugation length determines the energy gap:
- **Longer conjugation** → smaller gap → red-shifted emission
- **Shorter conjugation** → larger gap → blue-shifted emission

GFP's 11-atom conjugated system gives the perfect gap for green light!

**Quantum Yield**

GFP has a quantum yield (Φ) of ~0.79, meaning:

$$\Phi = \frac{\text{photons emitted}}{\text{photons absorbed}} \approx 0.79$$

This is exceptionally high - 79% of absorbed photons result in fluorescence!

---

## 1. The Starting Motif

We generate a peptide containing the critical `SER-TYR-GLY` motif. Before maturation, this is just a standard polypeptide backbone.


In [ ]:
sequence = "VAL-PRO-SER-TYR-GLY-VAL-LEU"

display(HTML("""
<div style='background: linear-gradient(135deg, #00C853 0%, #00796B 100%);
            color: white; padding: 15px; border-radius: 10px;
            font-family: monospace; margin-bottom: 15px;'>
    <b>🧬 GFP Chromophore Motif</b><br>
    Sequence: VAL-PRO-<span style='color: #FFD700;'>SER-TYR-GLY</span>-VAL-LEU<br>
    Critical residues: SER65, TYR66, GLY67<br>
    Status: Pre-maturation (non-fluorescent)
</div>
"""))

gen = PeptideGenerator(sequence)
peptide = gen.generate(conformation="alpha")
native_struct = peptide.structure.copy()

print("✅ Motif generated (pre-maturation state)")

## 2. The Maturation Process

The chromophore matures through three chemical transformations. While we can't show actual bond rearrangements in this simplified model, we'll highlight the critical residues at each stage.


In [ ]:
def get_step_info(step_name):
    """Return description and color for each maturation step."""
    info = {
        "1. Native": {
            "desc": "Standard polypeptide backbone. No fluorescence.",
            "bg_color": "#1a1a1a",
            "chemistry": "SER-TYR-GLY in linear configuration"
        },
        "2. Cyclization": {
            "desc": "Gly67 N attacks Ser65 C=O → 5-membered ring formed.",
            "bg_color": "#1a1a2a",
            "chemistry": "Nucleophilic addition-elimination"
        },
        "3. Dehydration": {
            "desc": "Water elimination from Tyr66 Cα → double bond created.",
            "bg_color": "#1a2a1a",
            "chemistry": "Cα=Cβ conjugation begins"
        },
        "4. Oxidation": {
            "desc": "O₂ oxidizes Tyr66 → full π-conjugation → FLUORESCENCE!",
            "bg_color": "#001a00",
            "chemistry": "Complete p-hydroxybenzylidene-imidazolinone"
        }
    }
    return info[step_name]

steps = ["1. Native", "2. Cyclization", "3. Dehydration", "4. Oxidation"]

## 3. The Interactive Molecular Forge

Use the slider to progress through the maturation steps. Watch the chromophore region (SER-TYR-GLY) highlighted in magenta.


In [ ]:
# Output widget for clean updates
out = widgets.Output()

# Slider
slider = SelectionSlider(options=steps, value=steps[0], description='Maturation:', layout=widgets.Layout(width='500px'))

# Track initialization
_initializing = True

def view_forge(change=None):
    global _initializing
    if _initializing and change is not None:
        return
    
    step = slider.value
    info = get_step_info(step)
    
    with out:
        clear_output(wait=True)
        
        # Display step information
        display(HTML(f"""
        <div style='background: linear-gradient(135deg, #00C853 0%, #00796B 100%);
                    color: white; padding: 15px; border-radius: 10px;
                    font-family: monospace; margin-bottom: 15px;'>
            <b>⚗️ {step}</b><br>
            {info['desc']}<br>
            <b>Chemistry:</b> {info['chemistry']}
        </div>
        """))
        
        # 3D viewer
        struct = native_struct.copy()
        
        # Convert to PDB
        temp = tempfile.NamedTemporaryFile(suffix=".pdb", delete=False)
        f = PDBFile()
        f.set_structure(struct)
        f.write(temp.name)
        with open(temp.name, 'r') as r:
            pdb_content = r.read()
        os.unlink(temp.name)
        
        view = py3Dmol.view(width=700, height=450)
        view.addModel(pdb_content, "pdb")
        
        # Style: Highlight chromophore motif
        view.setStyle({'resn': ['VAL', 'PRO', 'LEU']}, 
                     {'stick': {'color': 'gray', 'opacity': 0.4}})
        view.setStyle({'resn': ['SER', 'TYR', 'GLY']}, 
                     {'stick': {'colorscheme': 'magentaCarbon', 'radius': 0.25}})
        
        # Add glow effect for final step
        if step == "4. Oxidation":
            view.setBackgroundColor(info['bg_color'])
            view.addLabel("💚 FLUORESCENT!", 
                         {'position': {'x': 0, 'y': 5, 'z': 0}, 
                          'backgroundColor': '#00FF00', 
                          'fontColor': 'black',
                          'fontSize': 14})
        else:
            view.setBackgroundColor(info['bg_color'])
        
        view.zoomTo({'resn': ['SER', 'TYR', 'GLY']})
        display(view.show())

# Connect slider
slider.observe(view_forge, 'value')

# Display UI
display(widgets.VBox([slider, out]))

# Initialize
_initializing = False
view_forge()


## 4. Absorption & Emission Spectra

Visualizing the spectroscopic properties of mature GFP.


In [ ]:
# Create absorption and emission spectra
wavelengths = np.linspace(350, 600, 500)

# Gaussian approximations of GFP spectra
def gaussian(x, mu, sigma, amplitude):
    return amplitude * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

# Two absorption peaks
abs_395 = gaussian(wavelengths, 395, 15, 0.6)
abs_475 = gaussian(wavelengths, 475, 20, 1.0)
absorption = abs_395 + abs_475

# Emission peak
emission = gaussian(wavelengths, 508, 25, 0.79)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=wavelengths, y=absorption,
    name='Absorption',
    line=dict(color='blue', width=3),
    fill='tozeroy',
    fillcolor='rgba(0,0,255,0.2)'
))

fig.add_trace(go.Scatter(
    x=wavelengths, y=emission,
    name='Emission',
    line=dict(color='green', width=3),
    fill='tozeroy',
    fillcolor='rgba(0,255,0,0.2)'
))

fig.update_layout(
    title='GFP Absorption & Emission Spectra',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Normalized Intensity',
    template='plotly_dark',
    width=800, height=500,
    annotations=[
        dict(x=395, y=0.6, text="λ_abs = 395 nm", showarrow=True, arrowhead=2),
        dict(x=475, y=1.0, text="λ_abs = 475 nm", showarrow=True, arrowhead=2),
        dict(x=508, y=0.79, text="λ_em = 508 nm", showarrow=True, arrowhead=2)
    ]
)

fig.show()

---

## 🎓 Key Insights

1. **Autocatalytic Maturation**: GFP creates its own chromophore without external cofactors
2. **Three-Step Process**: Cyclization → Dehydration → Oxidation
3. **π-Conjugation**: Extended conjugated system creates the HOMO-LUMO gap for green emission
4. **High Quantum Yield**: 79% efficiency makes GFP an excellent fluorescent marker
5. **Irreversible**: Once formed, the chromophore is stable for the protein's lifetime

## 💡 Scientific Fact

The maturation process is **irreversible** and takes ~4 hours at 37°C. However, the β-barrel structure is critical - if it unfolds (e.g., high temperature), water quenching instantly destroys fluorescence!

## 📖 Further Reading

**Discovery & Development:**
- Shimomura et al. (1962). "Extraction, purification and properties of aequorin." *J Cell Comp Physiol* 59:223-239. [DOI: 10.1002/jcp.1030590302](https://doi.org/10.1002/jcp.1030590302)
- Chalfie et al. (1994). "Green fluorescent protein as a marker for gene expression." *Science* 263:802-805. [DOI: 10.1126/science.8303295](https://doi.org/10.1126/science.8303295)

**Structure & Mechanism:**
- Ormö et al. (1996). "Crystal structure of the Aequorea victoria green fluorescent protein." *Science* 273:1392-1395. [DOI: 10.1126/science.273.5280.1392](https://doi.org/10.1126/science.273.5280.1392)
- Heim et al. (1994). "Wavelength mutations and posttranslational autoxidation of green fluorescent protein." *PNAS* 91:12501-12504. [DOI: 10.1073/pnas.91.26.12501](https://doi.org/10.1073/pnas.91.26.12501)

**Spectroscopy:**
- Tsien, R.Y. (1998). "The green fluorescent protein." *Annu Rev Biochem* 67:509-544. [DOI: 10.1146/annurev.biochem.67.1.509](https://doi.org/10.1146/annurev.biochem.67.1.509)

**Nobel Prize:**
- The Nobel Prize in Chemistry 2008: [nobelprize.org/prizes/chemistry/2008/summary](https://www.nobelprize.org/prizes/chemistry/2008/summary/)

---

<div style='background: linear-gradient(135deg, #00C853 0%, #00796B 100%); padding: 20px; border-radius: 10px; color: white; text-align: center;'>
    <h3>🎉 Forge Session Complete!</h3>
    <p>You've witnessed the birth of biological light! 💚</p>
</div>